# Downloading files from Drive


In [0]:
#import statements
import cv2 #opencv
from google.colab import files #Not sure what this does, copied this stuff
import urllib.request #Downloads images from urls.
import os #for chdir and such
import pandas as pd #reading csv data and manipulating it.

In [0]:
#Clear out all filler data populated by colab upon creating environment
!rm -r /content/*

In [0]:
#Allowing drive permissions
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#Downloading the stuff
#CHECK THE LINK IS ACCURATE AND POINTS TO THE DESIRED FILES
urls_link = "https://drive.google.com/open?id=1FrWI-OzdwrEFUxzmMPl12Hp6XfU6iRo9" #URL file.  Maps name to url
fluff, urls_id = urls_link.split('=')

data_link = "https://drive.google.com/open?id=1XZr2j9WwiLll3Ww4VrhBYNez-dn_ecuE" #image data file.
fluff, data_id = data_link.split('=')

labels_link = "https://drive.google.com/open?id=1uHQi7XvLXNX7WLNhQ3j3WGhYxzX7zefo" #Gives us key of "LabelName" to actual object name. (i.e. "/m/01g317" -> "Human person" or whatever it is.)
fluff, labels_id = labels_link.split('=')

pos_zip_link = 'https://drive.google.com/open?id=1SGYYc_kCl8DmvKQ4O9sRD9spNv2dfoB1'
fluff, pos_zip_id = pos_zip_link.split('=')

pos_samples_link = 'https://drive.google.com/open?id=181Jcc_CayrSF7SnZdjL-6xYfC7ivmaf-'
fluff, pos_samples_id = pos_samples_link.split('=')

pos_txt_link = 'https://drive.google.com/open?id=1aI9Z69LUehNmAYBm_m7RfWH_StYbEfLn'
fluff, pos_txt_id = pos_txt_link.split('=')

neg_zip_link = 'https://drive.google.com/open?id=1aBJVH-NqbKlxhIhxwVf0UyNjhuAWBzYm'
fluff, neg_zip_id = neg_zip_link.split('=')

neg_txt_link = 'https://drive.google.com/open?id=1bVs30Bx2EZTJHH-EEhUpHlUNCSNtY95H'
fluff, neg_txt_id = neg_txt_link.split('=')

print(urls_id+"\n"+data_id+"\n"+labels_id+"\n"+pos_zip_id+"\n"+pos_txt_id+"\n"+neg_zip_id+"\n"+neg_txt_id)
#Verify the IDs match the bit after = in the above urls.

1FrWI-OzdwrEFUxzmMPl12Hp6XfU6iRo9
1XZr2j9WwiLll3Ww4VrhBYNez-dn_ecuE
1uHQi7XvLXNX7WLNhQ3j3WGhYxzX7zefo
1SGYYc_kCl8DmvKQ4O9sRD9spNv2dfoB1
1aI9Z69LUehNmAYBm_m7RfWH_StYbEfLn
1aBJVH-NqbKlxhIhxwVf0UyNjhuAWBzYm
1bVs30Bx2EZTJHH-EEhUpHlUNCSNtY95H


In [5]:
#Actually download the files

#print('Donloading URLs')
#downloaded = drive.CreateFile({'id':urls_id}) 
#downloaded.GetContentFile('urls.csv')

#print('Donloading Data')
#downloaded = drive.CreateFile({'id':data_id}) 
#downloaded.GetContentFile('data.csv')

#print('Donloading Labels')
#downloaded = drive.CreateFile({'id':labels_id}) 
#downloaded.GetContentFile('labels.csv')

print('Donloading pos base images')
downloaded = drive.CreateFile({'id':pos_zip_id}) 
downloaded.GetContentFile('pos.zip')

print('Downloading pos samples')
downloaded = drive.CreateFile({'id':pos_samples_id}) 
downloaded.GetContentFile('pos_samples.zip')

print('Downloading pos txt')
downloaded = drive.CreateFile({'id':pos_txt_id}) 
downloaded.GetContentFile('pos.txt')

print('Downloading neg images')
downloaded = drive.CreateFile({'id':neg_zip_id}) 
downloaded.GetContentFile('neg.zip')

print('Downloading neg txt')
downloaded = drive.CreateFile({'id':neg_txt_id}) 
downloaded.GetContentFile('neg.txt')

print('Downloads done!\n')
#The only files you sould see are "urls.csv", "data.csv", and "labels.csv".  The
#last one has something to do with the import procedure. #There may be "adc.json", which is irrelevant
!ls

Donloading pos base images
Downloads done!

adc.json  neg.txt  neg.zip  pos_samples.zip  pos.txt  pos.zip


In [6]:
#Unzip zipped files.
!mkdir base_images
!unzip pos.zip -d base_images
!unzip pos_samples.zip
!mkdir neg
!unzip neg.zip

Output hidden; open in https://colab.research.google.com to view.

25021


# Setting up Data
We imported 3 files above.
urls.csv maps the image url, from which we can download the image, to a name.  
Data.csv maps the name to the data about each image.  
Labels.csv lets us translate the Label from gibberish to English.


In [0]:
#Read in the data and store as dataframes
os.chdir("/content")
print("reading urls")
urls = pd.read_csv("urls.csv")
print("reading data")
data = pd.read_csv("data.csv")
print("reading labels")
labels = pd.read_csv("labels.csv",names=['Label','Name']) #Label is the auto-gen, Name is english.
print(urls.head())
print(data.head())
print(labels.head())

reading urls
reading data
reading labels
             image_name                                          image_url
0  e39871fd9fd74f55.jpg  https://requestor-proxy.figure-eight.com/figur...
1  f18b91585c4d3f3e.jpg  https://requestor-proxy.figure-eight.com/figur...
2  ede6e66b2fb59aab.jpg  https://requestor-proxy.figure-eight.com/figur...
3  ed600d57fcee4f94.jpg  https://requestor-proxy.figure-eight.com/figur...
4  ff47e649b23f446d.jpg  https://requestor-proxy.figure-eight.com/figur...
            ImageID  Source  LabelName  ...  IsGroupOf  IsDepiction  IsInside
0  000002b66c9c498e  xclick  /m/01g317  ...          0            0         0
1  000002b66c9c498e  xclick  /m/01g317  ...          0            0         0
2  000002b66c9c498e  xclick  /m/01g317  ...          0            0         0
3  000002b66c9c498e  xclick  /m/01g317  ...          0            0         0
4  000002b66c9c498e  xclick  /m/01g317  ...          0            0         0

[5 rows x 13 columns]
         Label    

In [0]:
#Find label corresponding to "Mobile phone"
phone_name = 'Mobile phone'
phone_label = labels[labels.Name == phone_name].iat[0,0]
#Check that the value is "/m/050k8".
print(phone_label)

/m/050k8


In [0]:
#Split data into phone and not phone
phone_data = data[data.LabelName == phone_label]

print(phone_data.head())
print(phone_data.shape)

               ImageID  Source LabelName  ...  IsGroupOf  IsDepiction  IsInside
1149  0000b9115cdf1e54  xclick  /m/050k8  ...          0            0         0
2487  00016982f6086d39  xclick  /m/050k8  ...          0            0         0
3963  0002a1a755d730ca  xclick  /m/050k8  ...          0            0         0
5408  000384bb6da4764b  xclick  /m/050k8  ...          0            0         0
8040  0005829bfcf77ca3  xclick  /m/050k8  ...          0            0         0

[5 rows x 13 columns]
(6365, 13)


In [0]:
#ENSURE THESE PATHS POINT TO THE APPROPRIATE FILES TO BE USED
pos_dir = 'pos'
neg_dir = 'neg'
num_samples = 2000
pos_txt_path = 'pos.txt'
neg_txt_path = 'neg.txt'

# Preparing Downloladed Data Files
We need a negative/background text file which has a bunch of filepaths to negative images.  Should be about half as many as positive images.
We need a positive file which has a bunch of "image filepath, num_Objects, x1,y1,x2,y2" lines


In [0]:
#Setting up directories for positive/negative files.
os.chdir("/content")
base_url = "https://requestor-proxy.figure-eight.com/figure_eight_datasets/open-images/train/"
height = 100  #Size of resized images.  Change if training time/acc suffers.
width = 100

In [0]:
#@MATT DO NOT RUN THIS BLOCK.  WE HAVE IMPORTED POS.TXT AND POS.ZIP ABOVE
#DO NOT RUN THIS IF POS.TXT AND POS.ZIP WERE IMPORTED ABOVE
%%time
#Download positive images from urls and save them locally, grayscaled and scaled down.
#NOTE this imports a few images per second, and imports a total of 6k images.
#Thus, it will take 30-60 mins, maybe even longer.
#TODO: Refactor code, make phone be 100x100 instead of the image.  Requires slightly more complicated math.
!rm pos.txt
!rm -r pos
!mkdir pos
pos_file = open('pos.txt','w')
base_url = "https://requestor-proxy.figure-eight.com/figure_eight_datasets/open-images/train/"
height = 100  #Size of resized images.  Change if training time/acc suffers.
width = 100
numpics = 0
for index, row in phone_data.iterrows():
    try:
        id = row['ImageID']
        url = base_url + id + ".jpg"
        path = "pos/"+ id +".jpg"
        print(numpics,id)
        #Avoid downloading pic if it's already here.
        #File doesn't exist yet, download it.
        if not os.path.exists(path):
            urllib.request.urlretrieve(url, path)   #Downloads img from url to path
            img = cv2.imread(path,cv2.IMREAD_GRAYSCALE) #Grayscales image
            resized_image = cv2.resize(img, (height, width))  #resizes image.  
            cv2.imwrite(path,resized_image)

        pos_file.write(path + " 1 " + str(int(row['XMin']*width)) + " " +   #Record the data to pos.txt
                       str(int(row["YMin"]*height)) + " " + str(int(row["XMax"]*width)) + " "  
                       + str(int(row["YMax"]*height)) + "\n")
        numpics += 1
        
    except Exception as e:
        print(str(e))
pos_file.close()
#print(phone_data.head())

NameError: ignored

In [0]:
#@MATT DO NOT RUN THIS BLOCK
#Save Pos Image data to file
#Only do this if you ran the above block and the results are satisfactory.
!zip pos.zip pos/*
#Upload pos data to drive
upload = drive.CreateFile({'title': 'pos.zip'})
upload.SetContentFile('pos.zip')
upload.Upload()

upload = drive.CreateFile({'title': 'pos.txt'})
upload.SetContentFile('pos.txt')
upload.Upload()

In [0]:
#@MATT DO NOT RUN THIS BLOCK.  NEG DATA HAS BEEN GENERATED AND IS IMPORTED ABOVE.
#Negative Data Generation.
#DO NOT RUN THIS IF NEG.TXT AND NEG.ZIP WERE IMPORTED ABOVE
%%time
!rm -r neg
!mkdir neg 
!rm neg.txt

#Get list of all ids of imgs with phones in them.
pos_set = set()
for filename in os.listdir('pos'):
    id, fluff = filename.split('.')
    pos_set.add(id)
#Find num pos examples in pos.txt
num_pos = 0
with open('pos.txt','r') as f:
    for line in f:
        num_pos += 1
print(num_pos)
#Gather num_pos/2 negative images that are not in pos_set
num_neg = 0
neg_file = open('neg.txt','w')
for index, row in urls.iterrows():
    id,fluff = row['image_name'].split('.')
    if id in pos_set:
        continue

    #image is not in positive set
    try:
        url = base_url + id + ".jpg"
        path = "neg/"+ id +".jpg"
        print(num_neg,id)
        #Avoid downloading pic if it's already here.
        #File is already downloaded, skip it.
        if os.path.exists(path):
            continue

        urllib.request.urlretrieve(url, path)   #Downloads img from url to path
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE) #Grayscales image
        resized_image = cv2.resize(img, (height, width))  #resizes image.  
        cv2.imwrite(path,resized_image)

        neg_file.write(path + "\n")
        num_neg += 1
        if num_neg >= num_pos/2:
            break
        
    except Exception as e:
        print(str(e))

neg_file.close()
#Sanity checks
print('pos examples:',num_pos)
print('neg examples:',len(os.listdir('neg')))
numnegim = 0
with open('neg.txt','r') as f:
    for line in f:
        numnegim += 1
print(numnegim)

In [0]:
#@MATT DO NOT RUN THIS CODE
#DO NOT RUN UNLESS PREVIOUS BLOCK WAS RUN AND DATA IS SATISFACTORY
#Zip and save the work because this took an hour and we're not doing this for every test.
#Only redo if we're changing something about the positive values.
!zip neg.zip neg/*
#Upload pos data to drive
upload = drive.CreateFile({'title': 'neg.zip'})
upload.SetContentFile('neg.zip')
upload.Upload()

upload = drive.CreateFile({'title': 'neg.txt'})
upload.SetContentFile('neg.txt')
upload.Upload()

# Preparing curated datasets.

Take a curated dataset and manipulate it so it is suitable for our purposes.

In [0]:
!cat pos.txt

/pos/0001_0059_0017_0031_0031.jpg 1 59 17 31 31
/pos/0001_0014_0054_0038_0038.jpg 1 14 54 38 38
/pos/0001_0027_0049_0043_0043.jpg 1 27 49 43 43
/pos/0001_0055_0025_0037_0037.jpg 1 55 25 37 37
/pos/0001_0053_0007_0034_0034.jpg 1 53 7 34 34
/pos/0001_0010_0023_0044_0044.jpg 1 10 23 44 44
/pos/0001_0011_0062_0026_0026.jpg 1 11 62 26 26
/pos/0001_0007_0015_0064_0064.jpg 1 7 15 64 64
/pos/0001_0012_0003_0065_0065.jpg 1 12 3 65 65
/pos/0001_0012_0026_0059_0059.jpg 1 12 26 59 59
/pos/0001_0018_0038_0054_0054.jpg 1 18 38 54 54
/pos/0001_0044_0056_0033_0033.jpg 1 44 56 33 33
/pos/0001_0030_0015_0043_0043.jpg 1 30 15 43 43
/pos/0001_0040_0034_0048_0048.jpg 1 40 34 48 48
/pos/0001_0049_0058_0027_0027.jpg 1 49 58 27 27
/pos/0001_0025_0013_0065_0065.jpg 1 25 13 65 65
/pos/0001_0018_0048_0045_0045.jpg 1 18 48 45 45
/pos/0001_0021_0047_0036_0036.jpg 1 21 47 36 36
/pos/0001_0045_0050_0039_0039.jpg 1 45 50 39 39
/pos/0002_0043_0018_0038_0038.jpg 1 43 18 38 38
/pos/0002_0020_0012_0024_0024.jpg 1 20 12 2

In [0]:
#Resize images to 100x100 grayscale
max_dim = 100

for path in os.listdir('base_images'):
    path = "base_images/"+path
    img = cv2.imread(path,cv2.IMREAD_GRAYSCALE) #Grayscales image
    #Resize image with aspect ratio preserved
    print(img.shape[0],img.shape[1])
    scale_factor = max_dim/max(img.shape[0],img.shape[1])
    height = int(img.shape[0] * scale_factor)
    width = int(img.shape[1] * scale_factor)
    assert(max(height,width) < 102)
    print(path, height, width)
    resized_image = cv2.resize(img, (height, width))  #resizes image.  

    cv2.imwrite(path,resized_image)


53 99
base_images/img23.jpg 53 100
50 100
base_images/img17.jpg 50 100
15 100
base_images/img12.jpg 15 100
54 100
base_images/img9.jpg 54 100
46 100
base_images/img4.jpg 46 100
49 99
base_images/img8.jpg 49 100
47 100
base_images/img22.jpg 47 100
53 100
base_images/img25.jpg 53 100
30 100
base_images/img27.jpg 30 100
52 100
base_images/img5.jpg 52 100
47 100
base_images/img14.jpg 47 100
58 100
base_images/img20.jpg 58 100
49 100
base_images/img2.jpg 49 100
28 100
base_images/img26.jpg 28 100
15 100
base_images/img24.jpg 15 100
50 100
base_images/img16.jpg 50 100
51 100
base_images/img1.jpg 51 100
16 100
base_images/img11.jpg 16 100
95 100
base_images/img18.jpg 95 100
51 100
base_images/img6.jpg 51 100
54 100
base_images/img3.jpg 54 100
77 100
base_images/img19.jpg 77 100
47 100
base_images/img13.jpg 47 100
64 100
base_images/img7.jpg 64 100
50 100
base_images/img21.jpg 50 100


In [0]:
#Takes base_images dir and runs createsamples on each
!mkdir pos
num_samples = 1000
for img in os.listdir('base_images'):
    path = 'base_images/' + img
    img_id, ignore = img.split('.')
    pos_path = "pos/" + img_id + ".lst"
    print(img_id,path,pos_path)
    !opencv_createsamples -bg /content/neg.txt -img $path -info $pos_path -pngoutput -num $num_samples

mkdir: cannot create directory ‘pos’: File exists
img23 base_images/img23.jpg pos/img23.lst
Info file name: pos/img23.lst
Img file name: base_images/img23.jpg
Vec file name: (NULL)
BG  file name: /content/neg.txt
Num: 1000
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 24
Height: 24
Max Scale: -1
Create test samples from single image applying distortions...
Open background image: /content/neg/e39871fd9fd74f55.jpg
Open background image: /content/neg/e7509670b9cedb8a.jpg
Open background image: /content/neg/f3a158f62532c925.jpg
Open background image: /content/neg/f88d1df66496556f.jpg
Open background image: /content/neg/e096ff9377f4c611.jpg
Open background image: /content/neg/fba3e2be74fea3d2.jpg
Open background image: /content/neg/e102801748581b7a.jpg
Open background image: /content/neg/e522b824327bb862.jpg
Open background image: /content/neg/fc7e7b807be78353.jpg
Open background image: /c

KeyboardInterrupt: ignored

In [0]:
#Takes all the .lst files and turns them into a single file.
#Builds the new file by alternating betwen all other files to ensure pics from
#each file are represented.

#Gets all .lst files and puts them in a list.
filelist = []
for file in os.listdir("pos"):
    if file.endswith(".lst"):
        filelist.append(open('pos/'+file,'r'))

outfile = open('pos.txt','w')
totallines = 0
fileiterations = 0
#Loops through all .lst files and cats them into pos.txt
while len(filelist) > 0:
    for f in filelist:
        line = f.readline()
        #If file empty, close and remove it.
        if line == '':
            f.close()
            filelist.remove(f)
            continue
        print(line)
        outfile.write(line)
        totallines += 1
    fileiterations += 1
        
outfile.close()
print(fileiterations,totallines)

In [0]:
!ls pos

In [0]:
#Zipping and uploading to drive, for easier future use.
!rm pos.zip
!zip pos.zip pos/*

upload = drive.CreateFile({'title': 'possamples.zip'})
upload.SetContentFile('pos.zip')
upload.Upload()

upload = drive.CreateFile({'title': 'pos.txt'})
upload.SetContentFile('pos.txt')
upload.Upload()

# Training the Cascade model.


In [12]:
pos_txt_path = '/content/pos/pos.txt'
os.rename('pos.txt','pos/pos.txt')
!mkdir data

FileNotFoundError: ignored

In [0]:
!rm data/*

In [10]:
!opencv_createsamples -bg /content/neg.txt -info $pos_txt_path  -vec output.vec -w 20 -h 20 -num 50000

Info file name: /content/pos/pos.txt
Img file name: (NULL)
Vec file name: output.vec
BG  file name: /content/neg.txt
Num: 50000
BG color: 0
BG threshold: 80
Invert: FALSE
Max intensity deviation: 40
Max x angle: 1.1
Max y angle: 1.1
Max z angle: 0.5
Show samples: FALSE
Width: 20
Height: 20
Max Scale: -1
Create training samples from images collection...
/content/pos/pos.txt(25001) : parse errorDone. Created 25000 samples


In [16]:
!opencv_traincascade -data /content/data -vec output.vec -bg /content/neg.txt -numPos 2000 -numNeg 1000 -numStages 15 -w 20 -h 20 -acceptanceRatioBreakValue 0.00005 -maxWeakCount 150 -minHitRate 0.7 -maxFalseAlarmRate 0.2

---------------------------------------------------------------------------------
Training parameters are pre-loaded from the parameter file in data folder!
Please empty this folder if you want to use a NEW set of training parameters.
---------------------------------------------------------------------------------
PARAMETERS:
cascadeDirName: /content/data
vecFileName: output.vec
bgFileName: /content/neg.txt
numPos: 2000
numNeg: 1000
numStages: 15
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : 5e-05
stageType: BOOST
featureType: HAAR
sampleWidth: 20
sampleHeight: 20
boostType: GAB
minHitRate: 0.7
maxFalseAlarmRate: 0.2
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 150
mode: BASIC
Number of unique features given windowSize [20,20] : 78460

Stages 0-6 are loaded

===== TRAINING 7-stage =====
<BEGIN
POS count : consumed   2000 : 19316
NEG count : acceptanceRatio    1000 : 1.77419e-05
The required acceptanceRatio for the model has been reached to avo

params.xml  stage1.xml	stage3.xml  stage5.xml	stage7.xml  stage9.xml
stage0.xml  stage2.xml	stage4.xml  stage6.xml	stage8.xml


In [0]:
!opencv_traincascade -data /content/data -vec output.vec -bg /content/neg.txt -numPos 2000 -numNeg 1000 -numStages 21 -w 20 -h 20 -maxFalseAlarmRate 0.1 -acceptanceRatioBreakValue 0.00001 -maxWeakCount 250

---------------------------------------------------------------------------------
Training parameters are pre-loaded from the parameter file in data folder!
Please empty this folder if you want to use a NEW set of training parameters.
---------------------------------------------------------------------------------
PARAMETERS:
cascadeDirName: /content/data
vecFileName: output.vec
bgFileName: /content/neg.txt
numPos: 2000
numNeg: 1000
numStages: 21
precalcValBufSize[Mb] : 1024
precalcIdxBufSize[Mb] : 1024
acceptanceRatioBreakValue : 1e-05
stageType: BOOST
featureType: HAAR
sampleWidth: 20
sampleHeight: 20
boostType: GAB
minHitRate: 0.995
maxFalseAlarmRate: 0.1
weightTrimRate: 0.95
maxDepth: 1
maxWeakCount: 250
mode: BASIC
Number of unique features given windowSize [20,20] : 78460

Stages 0-19 are loaded

===== TRAINING 20-stage =====
<BEGIN
POS count : consumed   2000 : 2401
NEG count : acceptanceRatio    1000 : 4.10575e-05
Precalculation time: 13
+----+---------+---------+
|  N |    HR

In [0]:
!opencv_traincascade -data data -vec output.vec -bg /content/neg.txt -numPos 2000 -numNeg 1000 -numStages 21 -w 20 -h 20 -maxFalseAlarmRate 0.2 -maxWeakCount 150

In [18]:
#!mv data/cascade.xml data/7stage70mhr20mfa.xml
!zip cascade.zip data/*

  adding: data/7stage70mhr20mfa.xml (deflated 82%)
  adding: data/params.xml (deflated 51%)
  adding: data/stage0.xml (deflated 68%)
  adding: data/stage1.xml (deflated 67%)
  adding: data/stage2.xml (deflated 65%)
  adding: data/stage3.xml (deflated 63%)
  adding: data/stage4.xml (deflated 60%)
  adding: data/stage5.xml (deflated 64%)
  adding: data/stage6.xml (deflated 61%)


In [0]:
upload = drive.CreateFile({'title': '7stage70mhr20mfa.zip'})
upload.SetContentFile('cascade.zip')
upload.Upload()